## Initialize

In [150]:

import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import requests
import pandas as pd
import os
import sys
from datetime import datetime
import base64

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['title', 'author', 'text', 'createdAt', 'changedAt']]
            df.rename(columns={'title': 'Title', 'author': 'Author',
                      'text': 'Text', 'createdAt': 'Created at', 'changedAt': 'Changed at'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Text search

In [148]:
so = esh.EshObject(
    count=True,
    top=3,
    wherefound=True,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW')
            )
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 2


,Title,Author,Text,Created at,Changed at
0,BMW_report,Not specified,JVBERi0xLjcNJeLjz9MNCjE1ODMgMCBvYmoNPDwvTGluZW...,2022-11-02,2022-11-02
1,MERCEDES_report,Mercedes-Benz Group AG,JVBERi0xLjYNJeLjz9MNCjQ5NjMgMCBvYmoNPDwvRmlsdG...,2022-03-08,2022-03-10


## Search for several text snippets

In [4]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW Finance')
            ),
            
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Finance'))
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 2


,Title,Text,Created at,Changed at
0,MERCEDES_report,JVBERi0xLjYNJeLjz9MNCjQ5NjMgMCBvYmoNPDwvRmlsdG...,2022-03-08,2022-03-10
1,BMW_report,JVBERi0xLjcNJeLjz9MNCjE1ODMgMCBvYmoNPDwvTGluZW...,2022-11-02,2022-11-02


## Search for explicit numbers

In [5]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='2,340', isPhrase=True)
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 1


,Title,Text,Created at,Changed at
0,AUDI_report,JVBERi0xLjcNJeLjz9MNCjE2MTYxIDAgb2JqDTw8L0xpbm...,2022-10-27,2022-11-07


## Search for author

In [62]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['author']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Daria')
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 1


,Title,Author,Text,Created at,Changed at
0,AUDI_report,"Fomenko, Daria (I/FU-23)",JVBERi0xLjcNJeLjz9MNCjE2MTYxIDAgb2JqDTw8L0xpbm...,2022-10-27,2022-11-07


## Search for date when the documents were created

In [117]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['createdAt']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='2022-10-27')
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 1


,Title,Author,Text,Created at,Changed at
0,AUDI_report,"Fomenko, Daria (I/FU-23)",JVBERi0xLjcNJeLjz9MNCjE2MTYxIDAgb2JqDTw8L0xpbm...,2022-10-27,2022-11-07


## Fuzzy Search

In [152]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='stretgy', # strategy
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

Total count: 4


,Title,Author,Text,Created at,Changed at
0,AUDI_report,"Fomenko, Daria (I/FU-23)",JVBERi0xLjcNJeLjz9MNCjE2MTYxIDAgb2JqDTw8L0xpbm...,2022-10-27,2022-11-07
1,VW_financial_services_report,Not specified,JVBERi0xLjcNJeLjz9MNCjg4OSAwIG9iag08PC9MaW5lYX...,2022-03-23,2022-04-06
2,PORSCHE_report,Not specified,JVBERi0xLjcNJeLjz9MNCjQxMTkgMCBvYmoNPDwvTGluZW...,2021-03-21,2021-03-22
3,MERCEDES_report,Mercedes-Benz Group AG,JVBERi0xLjYNJeLjz9MNCjQ5NjMgMCBvYmoNPDwvRmlsdG...,2022-03-08,2022-03-10


## Search for creation time ranges

In [151]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['createdAt']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[2022-10-01 2022-10-31]')
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 0
No results


## Search for explizit text in a file

In [154]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW Group und Daimler Mobility veräußern PARK NOW an EasyPark Group', isPhrase=True)
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 1


,Title,Author,Text,Created at,Changed at
0,MERCEDES_report,Mercedes-Benz Group AG,JVBERi0xLjYNJeLjz9MNCjQ5NjMgMCBvYmoNPDwvRmlsdG...,2022-03-08,2022-03-10
